# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [45]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys_vacation import gkey


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [46]:
weather_csv = "../WeatherPy/weatherdata.csv"
weather_df = pd.read_csv(weather_csv)
weather_df

,Unnamed: 0,lat,lon,city,clouds,country,date,humidity,max_temp,wind_sp
0,0,-46.19,168.86,mataura,72,NZ,1579921241,60,66.99,8.01
1,1,-41.41,175.52,waipawa,83,NZ,1579921369,60,75.20,19.46
2,2,-51.62,-69.22,rio gallegos,30,AR,1579921082,87,46.40,5.82
3,3,40.60,-124.16,fortuna,90,US,1579920999,87,57.00,4.85
4,4,71.69,128.87,tiksi,0,RU,1579921258,78,-31.70,5.95
...,...,...,...,...,...,...,...,...,...,...
539,539,-22.27,28.75,mathathane,2,BW,1579921682,69,69.75,6.73
540,540,-41.45,147.17,launceston,90,AU,1579921407,68,70.00,17.22
541,541,43.12,5.80,sanary-sur-mer,90,FR,1579921682,87,55.00,3.36
542,542,-12.04,-77.03,lima,100,PE,1579921502,73,77.00,13.87


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [47]:
gmaps.configure(api_key=gkey)
humidity = weather_df["humidity"]

In [48]:
locations = weather_df[["lat", "lon"]].astype(float)

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig



Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [143]:
vacation_df = weather_df.loc[(weather_df["max_temp"]>75)]
vacation_df = vacation_df.loc[(vacation_df["max_temp"]<86)]
vacation_df = vacation_df.loc[(vacation_df["wind_sp"]<10)]
vacation_df = vacation_df.loc[(vacation_df["clouds"] == 0)]
vacation_df

,Unnamed: 0,lat,lon,city,clouds,country,date,humidity,max_temp,wind_sp
185,185,-34.73,135.87,port lincoln,0,AU,1579921269,48,75.43,3.85
263,263,-2.67,141.30,vanimo,0,PG,1579921139,71,83.66,6.93
267,267,-34.65,-59.43,mercedes,0,AR,1579921659,73,80.01,1.99
380,380,-8.77,160.70,auki,0,SB,1579921281,72,85.10,8.41
501,501,-21.04,-57.87,fuerte olimpo,0,PY,1579921399,75,75.29,2.39


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [183]:
hotel_df = vacation_df
hotel_df["Hotel Name"] = ""
hotel_df

,Unnamed: 0,lat,lon,city,clouds,country,date,humidity,max_temp,wind_sp,Hotel Name
185,185,-34.73,135.87,port lincoln,0,AU,1579921269,48,75.43,3.85,
263,263,-2.67,141.30,vanimo,0,PG,1579921139,71,83.66,6.93,
267,267,-34.65,-59.43,mercedes,0,AR,1579921659,73,80.01,1.99,
380,380,-8.77,160.70,auki,0,SB,1579921281,72,85.10,8.41,
501,501,-21.04,-57.87,fuerte olimpo,0,PY,1579921399,75,75.29,2.39,


In [184]:
cities = hotel_df["city"]
target_coordinates = cities
#target_search = "Hotel"
target_radius = 5000
target_type = "lodging"
hotels = []

params = {
    #"keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": gkey
}


for index, row in hotel_df.iterrows():
    print(row["lat"], row["lon"])

    lat_var = row["lat"]
    lon_var = row["lon"]
    params["location"] = f"{lat_var},{lon_var}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    response = requests.get(base_url, params=params)
    places_data = response.json()
    
    try:
        print(places_data["results"][0]["name"])
        hotels.append(places_data["results"][0]["name"])
    
    except:
        pass

-34.73 135.87
Port Lincoln Hotel
-2.67 141.3
Vahmoneh Lodge Vanimo
-34.65 -59.43
Posada del Inti Centro
-8.77 160.7
Auki Motel
-21.04 -57.87
Hotel Fuerte Olimpo


In [146]:
hotels

['Port Lincoln Hotel',
 'Vahmoneh Lodge Vanimo',
 'Posada del Inti Centro',
 'Auki Motel',
 'Hotel Fuerte Olimpo']

In [185]:
hotel_df["Hotel Name"] = hotels

hotel_df

,Unnamed: 0,lat,lon,city,clouds,country,date,humidity,max_temp,wind_sp,Hotel Name
185,185,-34.73,135.87,port lincoln,0,AU,1579921269,48,75.43,3.85,Port Lincoln Hotel
263,263,-2.67,141.30,vanimo,0,PG,1579921139,71,83.66,6.93,Vahmoneh Lodge Vanimo
267,267,-34.65,-59.43,mercedes,0,AR,1579921659,73,80.01,1.99,Posada del Inti Centro
380,380,-8.77,160.70,auki,0,SB,1579921281,72,85.10,8.41,Auki Motel
501,501,-21.04,-57.87,fuerte olimpo,0,PY,1579921399,75,75.29,2.39,Hotel Fuerte Olimpo


In [186]:
hotel_df

,Unnamed: 0,lat,lon,city,clouds,country,date,humidity,max_temp,wind_sp,Hotel Name
185,185,-34.73,135.87,port lincoln,0,AU,1579921269,48,75.43,3.85,Port Lincoln Hotel
263,263,-2.67,141.30,vanimo,0,PG,1579921139,71,83.66,6.93,Vahmoneh Lodge Vanimo
267,267,-34.65,-59.43,mercedes,0,AR,1579921659,73,80.01,1.99,Posada del Inti Centro
380,380,-8.77,160.70,auki,0,SB,1579921281,72,85.10,8.41,Auki Motel
501,501,-21.04,-57.87,fuerte olimpo,0,PY,1579921399,75,75.29,2.39,Hotel Fuerte Olimpo


In [188]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "lon"]]

In [192]:
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=[f"Hotel: {hotel}" for hotel in hotels]
)


fig = gmaps.figure()
fig.add_layer(hotel_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [193]:
# Add marker layer ontop of heat map

fig = gmaps.figure()

fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)

fig


# Display Map

Figure(layout=FigureLayout(height='420px'))